# Deploy ML Model in OSS

In [1]:
import os
import logging

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

In [2]:
import requests
import json
import os

auth = (
    os.getenv('OSS_USER', 'admin'),
    os.getenv('OSS_PASSWORD', 'admin')
)

def make_requests(protocol, host, port, path, method, body=None, header=None):
    try:
        response = None
        url = f"{protocol}://{host}:{port}{path}"
        if method == 'GET':
            response = requests.get(url, headers=header, auth=auth, verify=False)
        if method == 'POST':
            response = requests.post(url, data=json.dumps(body), headers=header, auth=auth, verify=False)
        if method == 'PUT':
            response = requests.put(url, data=json.dumps(body), headers=header, auth=auth, verify=False)
        if method == 'DELETE':
            response = requests.delete(url, headers=header, auth=auth, verify=False)
        return response
    except Exception as ex:
        logger.error(f"exception: {str(ex)}")
        raise ex

In [3]:
protocol = 'https'
host = '13.58.172.230'
port = 9200

header = {
    'Content-Type': 'application/json'
}

# Arguments

In [4]:
MODEL_GROUP = "search_v1_model_group"
MODEL_GROUP_DESCRIPTION = "search_v1_model_group"
MODELS = [
    {
        "MODEL_NAME":"huggingface/sentence-transformers/msmarco-distilbert-base-tas-b",
        "MODEL_VERSION":"1.0.2",
        "MODEL_FORMAT":"TORCH_SCRIPT",
        "MODEL_TYPE": "dense"
    },
    {
        "MODEL_NAME":"amazon/neural-sparse/opensearch-neural-sparse-encoding-v1",
        "MODEL_VERSION":"1.0.1",
        "MODEL_FORMAT":"TORCH_SCRIPT",
        "MODEL_TYPE": "sparse"
    }
]

PIPELINE_NAME = "default_ingestion_pipeline"
FIELD_MAP = {
    "huggingface/sentence-transformers/msmarco-distilbert-base-tas-b": "bert_embeddings",
    "amazon/neural-sparse/opensearch-neural-sparse-encoding-v1": "oss_sparse_embeddings"
}

SEARCH_PIPELINE_NAME='default_search_pipeline'

# Set Cluster Settings

In [5]:
SET_CLUSTER_SETTINGS_PATH = "/_cluster/settings"
SET_CLUSTER_SETTINGS_BODY = {
  "persistent": {
    "plugins": {
      "ml_commons": {
        "only_run_on_ml_node": "false",
        "model_access_control_enabled": "true",
        "native_memory_threshold": "99"
      }
    }
  },
  "transient": {
    "plugins.ml_commons.model_access_control_enabled": "true"
  }
}

response = make_requests(protocol, host, port, SET_CLUSTER_SETTINGS_PATH, 'PUT', body=SET_CLUSTER_SETTINGS_BODY, header=header)
logger.info(f"SET_CLUSTER_SETTINGS response: {response.status_code} body: {response.json()}")

C:\Users\samriddha.chatterjee\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '13.58.172.230'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
INFO:__main__:SET_CLUSTER_SETTINGS response: 200 body: {'acknowledged': True, 'persistent': {'plugins': {'ml_commons': {'only_run_on_ml_node': 'false', 'model_access_control_enabled': 'true', 'native_memory_threshold': '99'}}}, 'transient': {'plugins': {'ml_commons': {'model_access_control_enabled': 'true'}}}}


# Register Model Group

In [6]:
REGISTER_MODEL_GROUP_PATH = "/_plugins/_ml/model_groups/_register"
REGISTER_MODEL_GROUP_BODY = {
  "name": MODEL_GROUP,
  "description": MODEL_GROUP_DESCRIPTION
}

response = make_requests(protocol, host, port, REGISTER_MODEL_GROUP_PATH, 'POST', body=REGISTER_MODEL_GROUP_BODY, header=header)
logger.info(f"REGISTER_MODEL_GROUP response: {response.status_code} body: {response.json()}")
MODEL_GROUP_ID = response.json()['model_group_id']

C:\Users\samriddha.chatterjee\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '13.58.172.230'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
INFO:__main__:REGISTER_MODEL_GROUP response: 200 body: {'model_group_id': 'w-qX6pgB1IaUAysXzGtT', 'status': 'CREATED'}


In [7]:
TEST_MODEL_GROUP_PATH = "/_plugins/_ml/model_groups/_search"
TEST_MODEL_GROUP_BODY = {
  "query": {
    "match": {
      "_id": MODEL_GROUP_ID
    }
  }
}
response = make_requests(protocol, host, port, TEST_MODEL_GROUP_PATH, 'POST', body=TEST_MODEL_GROUP_BODY, header=header)
logger.info(f"TEST_MODEL_GROUP response: {response.status_code} body: {response.json()}")

C:\Users\samriddha.chatterjee\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '13.58.172.230'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
INFO:__main__:TEST_MODEL_GROUP response: 200 body: {'took': 18, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': '.plugins-ml-model-group', '_id': 'w-qX6pgB1IaUAysXzGtT', '_version': 1, '_seq_no': 0, '_primary_term': 1, '_score': 1.0, '_source': {'owner': {'backend_roles': ['admin'], 'custom_attribute_names': [], 'roles': ['own_index', 'all_access'], 'name': 'admin', 'user_requested_tenant': None}, 'created_time': 1756282473436, 'access': 'private', 'latest_version': 0, 'last_updated_time': 1756282473436, 'nam

# Register Model

In [8]:
MODEL_TASK_IDS = []

for m in MODELS:
    REGISTER_MODEL_PATH = "/_plugins/_ml/models/_register"
    REGISTER_MODEL_BODY = {
      "name": m['MODEL_NAME'],
      "version": m['MODEL_VERSION'],
      "model_group_id": MODEL_GROUP_ID,
      "model_format": m['MODEL_FORMAT']
    }
    
    response = make_requests(protocol, host, port, REGISTER_MODEL_PATH, 'POST', body=REGISTER_MODEL_BODY, header=header)
    logger.info(f"REGISTER_MODEL response: {response.status_code} body: {response.json()}")
    MODEL_TASK_IDS.append(response.json()['task_id'])

logger.info(f"task ids {MODEL_TASK_IDS}")

C:\Users\samriddha.chatterjee\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '13.58.172.230'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
INFO:__main__:REGISTER_MODEL response: 200 body: {'task_id': 'xOqX6pgB1IaUAysX-mvR', 'status': 'CREATED'}
C:\Users\samriddha.chatterjee\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '13.58.172.230'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
INFO:__main__:REGISTER_MODEL response: 200 body: {'task_id': 'xeqX6pgB1IaUAysX-2tY', 'status': 'CREATED'}
INFO:__main__:task ids ['xOqX6pgB1IaUAysX-mvR', 'xeqX6pgB1IaUAysX-2tY']


In [12]:
MODEL_IDS = []
MODEL_ID_MAP = {}
for idx, t in enumerate(MODEL_TASK_IDS):
    TEST_MODEL_PATH = f"/_plugins/_ml/tasks/{t}"
    response = make_requests(protocol, host, port, TEST_MODEL_PATH, 'GET', header=header)
    logger.info(f"TEST_MODEL response: {response.status_code} body: {response.json()}")
    data = response.json()
    if 'state' in data and data['state'] == 'COMPLETED':
        MODEL_ID_MAP[data['model_id']] = MODELS[idx]['MODEL_NAME']
        MODEL_IDS.append(data['model_id'])
logger.info(f"model ids {MODEL_IDS}")

C:\Users\samriddha.chatterjee\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '13.58.172.230'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
INFO:__main__:TEST_MODEL response: 200 body: {'model_id': 'x-qX6pgB1IaUAysX_GtU', 'task_type': 'REGISTER_MODEL', 'function_name': 'TEXT_EMBEDDING', 'state': 'COMPLETED', 'worker_node': ['qf6OhLINQ-G5QBzY1n8YLQ'], 'create_time': 1756282485346, 'last_update_time': 1756282493836, 'is_async': True}
C:\Users\samriddha.chatterjee\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '13.58.172.230'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warni

# Deploy Models

In [13]:
MODEL_DEPLOY_TASK_IDS = []

for m in MODEL_IDS:
    DEPLOY_MODEL_PATH = f"/_plugins/_ml/models/{m}/_deploy"
    response = make_requests(protocol, host, port, DEPLOY_MODEL_PATH, 'POST', body={}, header=header)
    logger.info(f"DEPLOY_MODEL response: {response.status_code} body: {response.json()}")
    MODEL_DEPLOY_TASK_IDS.append(response.json()['task_id'])

logger.info(f"task ids {MODEL_DEPLOY_TASK_IDS}")

C:\Users\samriddha.chatterjee\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '13.58.172.230'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
INFO:__main__:DEPLOY_MODEL response: 200 body: {'task_id': 'yOqv6pgB1IaUAysXOGup', 'task_type': 'DEPLOY_MODEL', 'status': 'CREATED'}
C:\Users\samriddha.chatterjee\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '13.58.172.230'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
INFO:__main__:DEPLOY_MODEL response: 200 body: {'task_id': 'yeqv6pgB1IaUAysXOWtX', 'task_type': 'DEPLOY_MODEL', 'status': 'CREATED'}
INFO:__main__:task i

In [14]:
for t in MODEL_DEPLOY_TASK_IDS:
    TEST_MODEL_DEPLOY_TASK_PATH = f"/_plugins/_ml/tasks/{t}"
    response = make_requests(protocol, host, port, TEST_MODEL_DEPLOY_TASK_PATH, 'GET', header=header)
    logger.info(f"TEST_MODEL response: {response.status_code} body: {response.json()}")
    data = response.json()
    if 'state' in data and data['state'] == 'COMPLETED':
        logger.info(f"task id {t} is {data['state']}")

C:\Users\samriddha.chatterjee\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '13.58.172.230'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
INFO:__main__:TEST_MODEL response: 200 body: {'model_id': 'x-qX6pgB1IaUAysX_GtU', 'task_type': 'DEPLOY_MODEL', 'function_name': 'TEXT_EMBEDDING', 'state': 'COMPLETED', 'worker_node': ['qf6OhLINQ-G5QBzY1n8YLQ'], 'create_time': 1756284008616, 'last_update_time': 1756284015359, 'is_async': True}
INFO:__main__:task id yOqv6pgB1IaUAysXOGup is COMPLETED
C:\Users\samriddha.chatterjee\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '13.58.172.230'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs

# Deploy ReRanker Model

In [15]:
CROSS_ENCODER_MODELS = [
    {
        "MODEL_NAME":"huggingface/cross-encoders/ms-marco-MiniLM-L-6-v2",
        "MODEL_VERSION":"1.0.2",
        "MODEL_FORMAT":"TORCH_SCRIPT",
        "MODEL_TYPE": "cross-encoder"
    }
]

# Register Cross Encoder Models

In [16]:
CE_MODEL_TASK_IDS = []

for m in CROSS_ENCODER_MODELS:
    REGISTER_MODEL_PATH = "/_plugins/_ml/models/_register"
    REGISTER_MODEL_BODY = {
      "name": m['MODEL_NAME'],
      "version": m['MODEL_VERSION'],
      "model_group_id": MODEL_GROUP_ID,
      "model_format": m['MODEL_FORMAT']
    }
    
    response = make_requests(protocol, host, port, REGISTER_MODEL_PATH, 'POST', body=REGISTER_MODEL_BODY, header=header)
    logger.info(f"REGISTER_MODEL response: {response.status_code} body: {response.json()}")
    CE_MODEL_TASK_IDS.append(response.json()['task_id'])

logger.info(f"task ids {CE_MODEL_TASK_IDS}")

C:\Users\samriddha.chatterjee\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '13.58.172.230'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
INFO:__main__:REGISTER_MODEL response: 200 body: {'task_id': 'y-qv6pgB1IaUAysXjmsp', 'status': 'CREATED'}
INFO:__main__:task ids ['y-qv6pgB1IaUAysXjmsp']


In [18]:
CE_MODEL_IDS = []
for idx, t in enumerate(CE_MODEL_TASK_IDS):
    TEST_MODEL_PATH = f"/_plugins/_ml/tasks/{t}"
    response = make_requests(protocol, host, port, TEST_MODEL_PATH, 'GET', header=header)
    logger.info(f"TEST_MODEL response: {response.status_code} body: {response.json()}")
    data = response.json()
    if 'state' in data and data['state'] == 'COMPLETED':
        CE_MODEL_IDS.append(data['model_id'])
logger.info(f"model ids {CE_MODEL_IDS}")

C:\Users\samriddha.chatterjee\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '13.58.172.230'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
INFO:__main__:TEST_MODEL response: 200 body: {'model_id': 'zOqv6pgB1IaUAysXj2sC', 'task_type': 'REGISTER_MODEL', 'function_name': 'TEXT_SIMILARITY', 'state': 'COMPLETED', 'worker_node': ['qf6OhLINQ-G5QBzY1n8YLQ'], 'create_time': 1756284030504, 'last_update_time': 1756284035832, 'is_async': True}
INFO:__main__:model ids ['zOqv6pgB1IaUAysXj2sC']


# Deploy ReRanker Models

In [19]:
CE_MODEL_DEPLOY_TASK_IDS = []

for m in CE_MODEL_IDS:
    DEPLOY_MODEL_PATH = f"/_plugins/_ml/models/{m}/_deploy"
    response = make_requests(protocol, host, port, DEPLOY_MODEL_PATH, 'POST', body={}, header=header)
    logger.info(f"DEPLOY_MODEL response: {response.status_code} body: {response.json()}")
    CE_MODEL_DEPLOY_TASK_IDS.append(response.json()['task_id'])

logger.info(f"task ids {CE_MODEL_DEPLOY_TASK_IDS}")

C:\Users\samriddha.chatterjee\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '13.58.172.230'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
INFO:__main__:DEPLOY_MODEL response: 200 body: {'task_id': 'zeqv6pgB1IaUAysX0Gt5', 'task_type': 'DEPLOY_MODEL', 'status': 'CREATED'}
INFO:__main__:task ids ['zeqv6pgB1IaUAysX0Gt5']


In [20]:
for t in CE_MODEL_DEPLOY_TASK_IDS:
    TEST_MODEL_DEPLOY_TASK_PATH = f"/_plugins/_ml/tasks/{t}"
    response = make_requests(protocol, host, port, TEST_MODEL_DEPLOY_TASK_PATH, 'GET', header=header)
    logger.info(f"TEST_MODEL response: {response.status_code} body: {response.json()}")
    data = response.json()
    if 'state' in data and data['state'] == 'COMPLETED':
        logger.info(f"task id {t} is {data['state']}")

C:\Users\samriddha.chatterjee\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '13.58.172.230'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
INFO:__main__:TEST_MODEL response: 200 body: {'model_id': 'zOqv6pgB1IaUAysXj2sC', 'task_type': 'DEPLOY_MODEL', 'function_name': 'TEXT_SIMILARITY', 'state': 'COMPLETED', 'worker_node': ['qf6OhLINQ-G5QBzY1n8YLQ'], 'create_time': 1756284047481, 'last_update_time': 1756284048767, 'is_async': True}
INFO:__main__:task id zeqv6pgB1IaUAysX0Gt5 is COMPLETED


# Create Ingestion Pipeline

In [21]:
processors = []
if len(MODEL_IDS) > 0:
    for i, m in enumerate(MODEL_IDS):
        if MODELS[i]['MODEL_TYPE'] == 'dense':
            processors.append({
                "text_embedding": {
                    "model_id": m,
                    "field_map": {
                      "text": FIELD_MAP[MODEL_ID_MAP[m]]
                    }
              }
            })
        else:
            processors.append({
                "sparse_encoding": {
                    "model_id": m,
                    "field_map": {
                      "text": FIELD_MAP[MODEL_ID_MAP[m]]
                    }
              }
            })
CREATE_PIPELINE_BODY = {
    "description": PIPELINE_NAME,
    "processors": processors
}
CREATE_PIPELINE_PATH = F"/_ingest/pipeline/{PIPELINE_NAME}"
response = make_requests(protocol, host, port, CREATE_PIPELINE_PATH, 'PUT', body=CREATE_PIPELINE_BODY, header=header)
logger.info(f"CREATE_PIPELINE response: {response.status_code} body: {response.json()}")

C:\Users\samriddha.chatterjee\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '13.58.172.230'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
INFO:__main__:CREATE_PIPELINE response: 200 body: {'acknowledged': True}


In [22]:
GET_PIPELINE_PATH = F"/_ingest/pipeline"
response = make_requests(protocol, host, port, GET_PIPELINE_PATH, 'GET', header=header)
logger.info(f"GET_PIPELINE response: {response.status_code} body: {response.json()}")

C:\Users\samriddha.chatterjee\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '13.58.172.230'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
INFO:__main__:GET_PIPELINE response: 200 body: {'default_ingestion_pipeline': {'description': 'default_ingestion_pipeline', 'processors': [{'text_embedding': {'model_id': 'x-qX6pgB1IaUAysX_GtU', 'field_map': {'text': 'bert_embeddings'}}}, {'sparse_encoding': {'model_id': 'xuqX6pgB1IaUAysX_Gsa', 'field_map': {'text': 'oss_sparse_embeddings'}}}]}}


# Create Search Pipeline

In [23]:
response_processors = []
if len(CE_MODEL_IDS) > 0:
    for i, m in enumerate(CE_MODEL_IDS):
        response_processors.append({
                  "rerank": {
                    "ml_opensearch": {
                      "model_id": m
                    },
                    "context": {
                      "document_fields": [
                        "text"
                      ]
                    }
                  }
                })

request_processor = {}
if len(MODEL_IDS) > 0:
    for i, m in enumerate(MODEL_IDS):
        request_processor[FIELD_MAP[MODEL_ID_MAP[m]]]=m
request_processors = [
    {
        'neural_query_enricher': {
            'neural_field_default_id': request_processor
        }
    }
]
CREATE_SEARCH_PIPELINE_BODY = {
    "description": SEARCH_PIPELINE_NAME,
    "request_processors": request_processors,
    "response_processors": response_processors
}
SEARCH_CREATE_PIPELINE_PATH = F"/_search/pipeline/{SEARCH_PIPELINE_NAME}"
response = make_requests(protocol, host, port, SEARCH_CREATE_PIPELINE_PATH, 'PUT', body=CREATE_SEARCH_PIPELINE_BODY, header=header)
logger.info(f"CREATE_PIPELINE response: {response.status_code} body: {response.json()}")

C:\Users\samriddha.chatterjee\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '13.58.172.230'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
INFO:__main__:CREATE_PIPELINE response: 200 body: {'acknowledged': True}


In [24]:
GET_PIPELINE_PATH = F"/_search/pipeline"
response = make_requests(protocol, host, port, GET_PIPELINE_PATH, 'GET', header=header)
logger.info(f"GET_PIPELINE response: {response.status_code} body: {response.json()}")

C:\Users\samriddha.chatterjee\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '13.58.172.230'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
INFO:__main__:GET_PIPELINE response: 200 body: {'default_search_pipeline': {'description': 'default_search_pipeline', 'request_processors': [{'neural_query_enricher': {'neural_field_default_id': {'bert_embeddings': 'x-qX6pgB1IaUAysX_GtU', 'oss_sparse_embeddings': 'xuqX6pgB1IaUAysX_Gsa'}}}], 'response_processors': [{'rerank': {'ml_opensearch': {'model_id': 'zOqv6pgB1IaUAysXj2sC'}, 'context': {'document_fields': ['text']}}}]}}


# Cleanup

In [ ]:
import time
"""
Delete ingest pipeline
"""
response = make_requests(protocol, host, port, f"/_search/pipeline/{PIPELINE_NAME}", 'DELETE', header=header)
logger.info(f"response: {response.status_code} body: {response.json()}")

"""
Undeploy models
"""
for m in [*MODEL_IDS, *CE_MODEL_IDS]:
    response = make_requests(protocol, host, port, f"/_plugins/_ml/models/{m}/_undeploy", 'POST', body={}, header=header)
    logger.info(f"response: {response.status_code} body: {response.json()}")
    time.sleep(120)
    response = make_requests(protocol, host, port, f"/_plugins/_ml/models/{m}", 'DELETE', header=header)
    logger.info(f"response: {response.status_code} body: {response.json()}")

"""
Delete model Group
"""
response = make_requests(protocol, host, port, f"/_plugins/_ml/model_groups/{MODEL_GROUP_ID}", 'DELETE', header=header)
logger.info(f"response: {response.status_code} body: {response.json()}")